In [1]:
import json
from pathlib import Path
import random
from PIL import Image

In [2]:
SRC_DIR = Path("/home/aikusrv01/storyboard/SY/Dataset/close-shot")
JSONL_PATH =  Path("/home/aikusrv01/storyboard/SY/Dataset/matched.jsonl")
OUT_JSONL = Path("./aa.jsonl")

# 1) 이미지 파일 확장자 목록
IMG_EXT = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".webp", ".JPG", ".JPEG"}

In [3]:
def load_jsonl(path: Path):
    """JSONL 파일을 읽어 list[dict] 로 반환"""
    data = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            data.append(json.loads(line))
    return data

In [4]:
def extract_json_by_images():
    # 1) 이미지 파일 목록 읽기
    image_files = {p.name for p in SRC_DIR.iterdir() if p.suffix in IMG_EXT}

    # 2) 전체 JSONL 로딩
    json_data = load_jsonl(JSONL_PATH)

    # 3) 이미지와 매칭되는 JSON만 필터링
    matched = [item for item in json_data if item["file_name"] in image_files]

    return matched

In [5]:
def save_matched_jsonl(new_data, out_path: Path):
    """기존 matched.jsonl과 중복 없이 병합 후 저장"""
    # 기존 파일 로드
    old_data = load_jsonl(out_path)

    # 이미 있는 file_name은 제외하여 신규만 추가
    existing_names = {item["file_name"] for item in old_data}

    to_append = [item for item in new_data if item["file_name"] not in existing_names]  

    # 신규가 없다면 종료
    if not to_append:
        print("추가할 새로운 데이터 없음.")
        return

    # append 방식으로 저장
    with out_path.open("a", encoding="utf-8") as f:
        for item in to_append:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"{len(to_append)}개 줄이 {out_path.name} 에 추가됨.")

In [6]:
if __name__ == "__main__":
    matched_data = extract_json_by_images()
    save_matched_jsonl(matched_data, OUT_JSONL)

추가할 새로운 데이터 없음.


In [7]:
import json

def add_shot_trigger(jsonl_path, output_path):
    def detect_shot(text):
        t = text.lower()
        if "full shot" in t:
            return "<fs_trg>"
        if "medium shot" in t:
            return "<ms_trg>"
        if "close-up shot" in t or "close up shot" in t:
            return "<cu_trg>"
        return None

    with open(jsonl_path, "r", encoding="utf-8") as f, \
         open(output_path, "w", encoding="utf-8") as out:
        
        for line in f:
            item = json.loads(line)
            text = item.get("text", "")

            trg = detect_shot(text)
            if trg:
                # 이미 붙어 있으면 중복 추가 방지
                if not text.strip().startswith(trg):
                    item["text"] = f"{trg}, {text}"

            out.write(json.dumps(item, ensure_ascii=False) + "\n")

    print("완료됨:", output_path)
    add_shot_trigger("aa.jsonl", "aa.jsonl")

In [8]:
def save_jsonl(path, data):
    path = Path(path)
    with path.open("w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [9]:
# 텍스트에서 left <-> right

def flip_text_label(text):
    text = text.replace("left", "__TEMP__")
    text = text.replace("right", "left")
    text = text.replace("__TEMP__", "right")
    # Left / Right 대문자 버전도 하고 싶으면 아래 추가 가능
    return text

# 이미지 Flip

def flip_image(img_path: Path, output_path : Path):
    img = Image.open(img_path)
    flipped = img.transpose(Image.FLIP_LEFT_RIGHT)
    flipped.save(output_path) 
    img.close()

def random_flip_images(jsonl_path):
    data = load_jsonl(jsonl_path)

    # full shot 또는 close-up 포함된 항목 전부 flip
    targets = [
        item for item in data
        if ("full shot" in item["text"]) or ("close-up" in item["text"])
    ]

    output_dir = Path("/home/aikusrv01/storyboard/SY/Dataset/flipped_image/")
    output_dir.mkdir(parents=True, exist_ok=True)

    new_entries = []

    for item in targets:
        filename = item["file_name"]
        img_path = SRC_DIR / filename

        if not img_path.exists():
            continue

        stem = img_path.stem
        suffix = img_path.suffix
        new_file_name = f"{stem}_flip{suffix}"

        output_path = output_dir / new_file_name

        # flip 이미지 저장
        flip_image(img_path, output_path)

        # 텍스트 left ↔ right 치환
        flipped_text = flip_text_label(item["text"])

        # flip된 json entry 추가
        new_entries.append({
            "file_name": new_file_name,
            "text": flipped_text
        })

    # 기존 JSON + flip된 JSON 합쳐서 저장
    all_data = data + new_entries
    save_jsonl(jsonl_path, all_data)

    print(f"{len(new_entries)}개의 full shot/close-up 이미지 flip 완료.")


In [11]:
# 실행
if __name__ == "__main__":
    MATCHED_JSONL = Path("./aa.jsonl")
    random_flip_images(MATCHED_JSONL)

140개의 full shot/close-up 이미지 flip 완료.
